### Convolutional Neural Networks from Scratch

#### What is a Convolution?

#### Convolution and Cross-Correlation 

In 2D convolution which we will be using to establish our CNN, the input matrix (which will eventually be out image) is 'convolved' with another matrix of smaller size called the 'Kernel.' The convolution between the input and the kernel, produces another matrix computed by sliding the Kernel matrix across the rows of the input matrix. Therefore, a convolution between two matrices $I$ and $K$ as shown below, can be demonstrated using the visualisation below. It is important to note that the kernel, $K$ has been 'rotated' by 180 degrees: this stems from the definition of convolution. If this rotation was not made, the resultant output would be the 'Cross-Correlation' instead. '*' refers to Convolution and '⭑' refers to Cross-Correlation or Correlation for short. 

$$
\begin{bmatrix}
1 & 6 & 2 \\
5 & 3 & 1 \\
7 & 0 & 4
\end{bmatrix} \ast 
\begin{bmatrix}
2 & 0 \\
1 & -1 \\
\end{bmatrix}
$$

> The size the output matrix, O, can be correlated with the size of the input, I, and the size of the kernel, K as $$O=I-K+1$$ 



In [191]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display
import matplotlib.patches as patches

# Define the matrices
input_matrix = np.array([[1, 6, 2],
                        [5, 3, 1],
                        [7, 0, 4]])

kernel = np.array([[0, -1],
                  [2, 1]])

output = np.array([[7, 5],
                  [11, 3]])

# Create figure and axes with smaller size
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(5, 1.5), dpi=200)
plt.subplots_adjust(wspace=0.5)

def draw_matrix(ax, matrix, title, highlight=None, frame_num=None):
    ax.clear()
    ax.set_title(title, fontsize=8)
    
    # Show the matrix with white background
    im = ax.imshow(matrix, vmin=np.min(matrix)-1, vmax=np.max(matrix)+1)
    
    # Add text annotations
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            # For output matrix, only show values up to current frame
            if title == 'Output' and frame_num is not None:
                current_pos = frame_num
                current_row = current_pos // 2
                current_col = current_pos % 2
                
                if (i > current_row) or (i == current_row and j > current_col):
                    text = ''
                else:
                    text = str(matrix[i, j])
            else:
                text = str(matrix[i, j])
                
            ax.text(j, i, text, ha='center', va='center', color='black', fontsize=6)
    
    # Add grid
    ax.set_xticks(np.arange(-.5, matrix.shape[1], 1), minor=True)
    ax.set_yticks(np.arange(-.5, matrix.shape[0], 1), minor=True)
    ax.grid(True, which='minor', color='gray', linewidth=1)
    
    if highlight is not None:
        rect = patches.Rectangle((highlight[1]-0.5, highlight[0]-0.5), 
                               2, 2, linewidth=2.5, 
                               edgecolor='red', facecolor='none')
        ax.add_patch(rect)
    
    ax.set_xticks([])
    ax.set_yticks([])


def update(frame):
    row = frame // 2
    col = frame % 2
    
    # Draw matrices
    draw_matrix(ax1, input_matrix, 'Input', highlight=(row, col))
    draw_matrix(ax2, kernel, 'Kernel')
    draw_matrix(ax3, output, 'Output', frame_num=frame)
    
    # Add multiplication and equals signs
    ax1.text(1.2, 0.5, '  *  ', transform=ax1.transAxes, 
             ha='center', va='center', fontsize=6)
    ax2.text(1.2, 0.5, '  =  ', transform=ax2.transAxes, 
             ha='center', va='center', fontsize=6)

# Create animation
anim = FuncAnimation(fig, update, frames=4, interval=1500, repeat=True)

# Display the animation
display(HTML(anim.to_jshtml()))
plt.close()

> Besides the input-modification, Convolution and Correlation are mathemtically equivalent. In case of CNNs, the algorithm doesn't care what the input kernel is, having to rotate it adds an extra step which we don't care about; so from here on we'll primarily be working with Correlation or mathematical Correlation, and update the weights that actually matter when needed. Instead of communicating in terms of the input kernel we'll instead be concerned only with the transformed kernel. 

It is also important to note that the convolution above is valid, meaning we stop if the kernel goes beyond the edge. In a full convolution wee would start off multiplying (2,2) of the Kernel with (1, 1) of the input; the rest of the kernel would be outside and the corresponding dotproducts between the values would be 0. Consequently the final output of a full convolution would be larger than the Input. 

#### Building a Convolution Layer

The convolutional layer takes in a 3 dimensional block od data (r, g, b values) as inputs, or the depth of the input is 3. The trainable parameters of the layer is are the kernels, *which span the depth of the input*, i.e each kernel has to be of depth 3. The layer may have multiple kernels, each of which also has a bias matrix of the size of the kernel. 

<center><img src="images/convolution-layer.png" width="450"></center>

To find $y_{11}^1$ for example, corresponding to the output of convoltion using the first kernel: 

$$y_{11}^1 = k^1 . x^1_{(1, 1), (2, 2)} + k^2 . x^2_{(1, 1), (2, 2)} + k^3 . x^3_{(1, 1), (2, 2)} + b_{11}^1$$

And so on. Abandoning the granularity, 

$$Y_1 = B_1 + \sum_{j=1}^3 X_j * K_j$$

To generalise even further, the input space doesn't need to be limited to 3 dimensions. Let the input be n-dimensional, and there be d different kernals for this layer, then the output for each kernel can be expressed as: 

$$Y_i = B_i + \sum_{j=1}^n X_j * K_{i, j}\;; \hspace{0.2cm} i = 1, 2, ..., d$$

This demonstrates forward propagation in a Convolutional Layer. Using this we can implement a Convolution class. 

In [192]:
from scipy import signal

class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        # Return output
        pass

    def backward(self, output_gradient, learning_rate):
        # Update parameters and return input gradient
        pass

class Convolution(Layer): 
    def __init__(self, input_shape, kernel_size, depth): 
        """
        input_shape: tuple of (depth, height, width) of the input
        kernel_size: int representing the size of each matrix, inside each kernel
        depth: int representing the number of kernels, and therefore the depth of the output
        """
        input_depth, input_height, input_width = input_shape # (depth, height, width)
        self.depth = depth
        self.input_shape = input_shape
        self.input_depth = input_depth
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1) # Depth numnber of outputs
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size) # Depth number of kernels, each with input_depth number of 
        # sq. matrices, of size kernel_size x kernel_size
        self.kernels = np.random.randn(*self.kernels_shape) # random kernels
        self.bias = np.random.rand(*self.output_shape) # random bias for each output
    
    def forward(self, input):
        self.input = input
        self.output = np.zeros(self.output_shape) 
        for i in range(self.depth): # For each kernel
            for j in range(self.input_depth): # input_depth = kernel_depth, for each matrix in the kernel 
                self.output[i] += signal.correlate2d(input[j], self.kernels[i, j], mode='valid') # For the i-th kernel, the output is the 
                # Sum of the correlation of each input matrix with the corresponding kernel matrix
            self.output[i] += self.bias[i, j]

    def backward(self, output_gradient, learning_rate):
        kernel_gradient = np.zeros(self.kernels_shape)
        input_gradient = np.zeros(self.input_shape)

        # Backpropagation
        for i in range(self.depth): # For each kernel
            for j in range(self.input_depth): # For each image in the input 
                kernel_gradient[i, j] = signal.correlate2d(input[j], output_gradient[i], mode='valid')
                input_gradient[j] += signal.convolve2d(output_gradient[i], self.kernels[i, j], mode='full')
        
        self.kernels -= learning_rate * kernel_gradient 
        self.bias -= learning_rate * output_gradient
        return input_gradient # We return the input gradient to pass it to the previous layer as its output gradient in backpropagation

#### Implementing Back Propagation

To implement BP, we first find the loss. We know $ \frac{\partial L}{\partial Y_i} $, and we want to find the gradient of every parameter, namely kernel, bias, and inputs. To find $ \frac{\partial L}{\partial K_{i, j}} $, 

$$Y_i = B_i + \sum_{j=1}^n X_j * K_{i, j}\;; \hspace{0.2cm} i = 1, 2, ..., d$$

Then from the equation of forward pass, 
$$ \frac{\partial L}{\partial K_{i, j}} = \frac{\partial L}{\partial Y_i}  \frac{\partial Y_i}{\partial K_{i, j}} = X_j * \frac{\partial L}{\partial Y_i}$$ 

Similarly, 
$$\frac{\partial L}{\partial B_i} = \frac{\partial L}{\partial Y_i} \frac{\partial Y_i}{\partial B_i} = \frac{\partial L}{\partial Y_i}$$

And finally, 
$$\frac{\partial L}{\partial Y_i} = \sum_{i=1}^d \frac{\partial L}{\partial Y_i} \underset{\text{full}}{\ast} rot180(K_{i, j}) \;; \hspace{0.2cm} j = 1, 2, 3 ..., d$$

Here $\underset{\text{full}}{\ast}$ refers to a full convoluton. 

#### Reshape Layer

Output of a Convolutional Layer is 3 dimensional, so to make predictions we need to convert this to a column vector, i.e dimension 1, and then pass it through a Dense Net. This conversion is done by the Reshape Layer. 

In [193]:
class Reshape(Layer):
    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape
    
    def forward(self, input):
        return np.reshape(input, self.output_shape)
    
    def backward(self, output_gradient, learning_rate):
        return np.reshape(output_gradient, self.input_shape) # Converts the output back to input shape for backpropagation

#### Dense Layer 

After the Reshape Layer, we use a Dense Layer to finally make predictions. 

In [194]:
class Dense(Layer):
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        self.weights = np.random.randn(output_size, input_size)
        self.bias = np.random.rand(output_size)
    
    def forward(self, input):
        self.input = input
        return np.dot(self.weights, input) + self.bias
    
    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient, self.input.T)
        input_gradient = np.dot(output_gradient, self.weights.T)
        # Bias gradient = output gradient
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * output_gradient
        return input_gradient

#### Binary-Cross Entropy Loss and Sigmoid Activation

In [195]:
def binary_cross_entropy(y_true, y_pred):
    return np.mean(-y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred))

def binary_cross_entropy_prime(y_true, y_pred):
    return ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)

class Activation(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input):
        self.input = input
        return self.activation(input)

    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient, self.activation_prime(self.input))
    
class Sigmoid(Activation):
    def __init__(self):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x))

        def sigmoid_prime(x):
            s = sigmoid(x)
            return s * (1 - s)

        super().__init__(sigmoid, sigmoid_prime)

#### MNIST: Binary Classification of 0 and 1

In [196]:
from keras.datasets import mnist
from keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = mnist.load_data()

def pp(x, y, lim):
    zeroi = np.where(y == 0)[0][:lim]
    onei = np.where(y == 1)[0][:lim]
    indices = np.hstack((zeroi, onei))
    np.random.shuffle(indices)
    x, y = x[indices], y[indices]
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype('float32')/255
    y = to_categorical(y)
    y = y.reshape(len(y), 2, 1)
    return x, y

x_train, y_train = pp(x_train, y_train, 1000)
x_test, y_test = pp(x_test, y_test, 100)

In [197]:
n = [
    Convolution((1, 28, 28), 3, 5),
    Sigmoid(),
    Convolution((5, 26, 26), 3, 5),
    Sigmoid(),
    Reshape((5, 24, 24), 600),
    Dense(600, 100),
    Sigmoid(),
    Dense(100, 2),
    Sigmoid()
]

In [198]:
epoch = 20; lr = 0.1

for e in range(epoch):
    loss = 0

    for x, y in zip(x_train, y_train):
        # Forward
        output = x
        for layer in n:
            output = layer.forward(output)

        loss += binary_cross_entropy(y, output)

        # Backward
        grad = binary_cross_entropy_prime(y, output)
        for layer in reversed(n):
            grad = layer.backward(grad, lr)
        
    loss /= len(x_train)
    print(f"{e+1}/{epoch}; loss: {loss}")

TypeError: bad operand type for unary -: 'NoneType'